# Workshop 1 - TF-IDF

This notebook implements TF-IDF from ground up to build the represent the word/phrase/sentence/paragraph by vector semantic.<br/>

<b>Author: </b> Arpit Gole <br/>
<b>Contact on: </b> <a href="mailto:arpit.gole@adelaide.edu.au">arpit.gole@adelaide.edu.au</a> <br/>
<b>Created on: </b> 27/08/22 <br/>

In [1]:
# Imports

import os
from tqdm import tqdm
from collections import Counter, defaultdict
import math

import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import  word_tokenize 

In [2]:
# Notebook configs 
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arpit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Reading/Generating the data

In [3]:
# Function to read the data
def read_txt(file):
    with open(file, 'r', encoding="utf8") as fp:
        return fp.readlines()

In [4]:
%%time

# Let's check if the data is already present
if not os.path.exists('medium_doc.txt') or not os.path.exists('large_doc.txt'):
    
    # Generating the data - essentially duplicating the original text n times
    for name, loop_num in [('medium_doc.txt', 10), ('large_doc.txt', 25)]:
        for _ in tqdm(range(loop_num), desc=f"Generating data for {name}"):
            with open(name, 'a+', encoding="utf8") as fp:
                fp.writelines(text)

CPU times: total: 0 ns
Wall time: 0 ns


In [5]:
# Reading the data
# Different files to read are:
# 1. doc.txt
# 2. medium_doc.txt
# 3. large_doc.txt
text = read_txt('doc.txt')

## Data Preprocessing

In [6]:
%%time

#Preprocessing the text data
sentences = []
 
for sent in text:
    x = [i.lower() for  i in word_tokenize(sent) if i.isalpha()]
    sentences.append(x)

# Removing the empty lines (which are read as empty sentences)
sentences = [sentence for sentence in sentences if len(sentence) != 0]

CPU times: total: 14.1 s
Wall time: 14.1 s


<b>Note:</b> Here, we are considering each sentence as a document. Whereas, in real world problem each document can a complete text file, movie review, basically any corpus of text.

In [7]:
# Creating a series.
X_train = pd.Series(sentences, dtype=object)

In [8]:
# Data visualisation
X_train.head(5)

0    [the, project, gutenberg, ebook, of, the, holl...
1    [this, ebook, is, for, the, use, of, anyone, a...
2    [most, other, parts, of, the, world, at, no, c...
3    [whatsoever, you, may, copy, it, give, it, awa...
4    [of, the, project, gutenberg, license, include...
dtype: object

## TF-IDF Algorithm using Inverted File.

In [9]:
%%time

# TF-IDF Implementation using Inverted File.

# Total number of documents in the set.
N = len(X_train)

# Global storing variables
tf = []
idf = {}
inverted_file = defaultdict(list)
vocab = set()

# TF Part
for doc_num, val in tqdm(zip(X_train.index, X_train), desc="Computing TF part"):
    temp = {}

    for k, v in Counter(val).items():
        # Formula given in the slides
        temp[k] = 1 + math.log10(v)
        vocab.add(k)

    tf.append((doc_num, temp))

# IDF part
for word in tqdm(vocab, desc="Computing IDF part"):
    word_doc_count = 0

    for val in X_train:
        if word in val:
            word_doc_count += 1
    
    # Formula given in the slides
    idf[word] = N / word_doc_count

# Putting it together - building Inverted File
for doc_num, val in tqdm(tf, desc="Computing Inverted File using TF-IDF"):
    for k, v in val.items():
        tf_idf = v * idf[k]
        inverted_file[k].append((doc_num, tf_idf))

Computing TF part: 156151it [00:00, 167926.20it/s]
Computing Inverted File using TF-IDF: 100%|████████████████████████████████| 156151/156151 [00:00<00:00, 337923.16it/s]

CPU times: total: 1min 2s
Wall time: 1min 2s


## Searching a word in the documents.

In [10]:
search_word = 'hollow'

In [11]:
%%time

# 1. Using the normal way using a loop. 
# Time complexity is O(n).
len([sentence for sentence in sentences if search_word in sentence])

CPU times: total: 31.2 ms
Wall time: 27.9 ms


1800

In [12]:
%%time

# 2. Using te TF-IDF.
# Time complexity is O(1).
docs_priority = defaultdict(int)

inverted_file_values = inverted_file[search_word]
                
if inverted_file_values != []:
    for doc_num, tfidf in inverted_file_values:

        docs_priority[doc_num] += tfidf   # Update the values 
else:
    print('Word is not present in any document') 

CPU times: total: 0 ns
Wall time: 1.01 ms


THE END